### Install Library

In [ ]:
pip install huggingface_hub==0.23.4 langchain==0.2.5 langchain_community==0.2.5 torch==2.3.0+cu121 transformers==4.40.1 pypdf==4.2.0

In [ ]:
pip install sentence-transformers==3.0.1 chromadb==0.5.3

In [ ]:
!pip install -U langchain-openai langchain_community pypdf

In [ ]:
!pip install -q --upgrade transformers huggingface_hub


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# 파일 및 정규표현식 관련 모듈
import os  # 운영 체제와 상호작용하기 위한 모듈
import re  # 정규 표현식을 사용하기 위한 모듈

# PyTorch 관련 모듈
import torch  # PyTorch 프레임워크

# Transformers 관련 모듈
from transformers import (
    AutoModelForCausalLM,  # 언어 모델을 로드하는 함수
    AutoTokenizer,  # 토크나이저를 로드하는 함수
    BitsAndBytesConfig,  # 양자화 및 압축을 위한 설정
    TrainingArguments,  # 모델 훈련을 위한 인자 설정
    pipeline,  # 파이프라인을 생성하는 함수
    logging,  # 로깅을 위한 함수
    AutoConfig,  # 설정을 로드하는 함수
    GenerationConfig  # 텍스트 생성을 위한 설정
)

# Langchain 관련 모듈
from langchain_community.document_loaders import PyPDFLoader  # PDF 문서 로더
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain.embeddings import HuggingFaceEmbeddings  # 임베딩 생성기
from langchain.schema import Document  # 문서 스키마
from langchain_community.vectorstores import Chroma  # 벡터 저장소
from langchain.chains.question_answering import load_qa_chain  # 질의 응답 체인 로드
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain  # 질의 응답 체인
from langchain.memory import ConversationSummaryMemory  # Langchain 메모리 관련 모듈대화 요약 메모리
from langchain.text_splitter import CharacterTextSplitter  # Langchain 텍스트 분할기 모듈 문자 기반 텍스트 분할기
from langchain.prompts import PromptTemplate  # Langchain 프롬프트 템플릿 관련 모듈 프롬프트 템플릿
from langchain.llms import HuggingFacePipeline  # Langchain LLM 관련 모듈 HuggingFace 파이프라인을 사용하는 LLM

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 2


In [ ]:
# In Case of Using CoLab
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
ls

In [ ]:
# In Case of Using CoLab
%cd /content/gdrive/MyDrive/LLMClass/rag

# 1. 모델 불러오기

In [5]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    force_download=True,
)


Exception: data did not match any variant of untagged enum ModelWrapper at line 1251003 column 3

# 2. 문서 불러오기

In [ ]:
loader = PyPDFLoader('/content/gdrive/MyDrive/LLMClass/ragp/info.pdf')
documents = loader.load()

# 3. 문서 분리(조각)

In [ ]:
output = []
# text 정제
for page in documents:
    text = page.page_content
    output.append(text)

doc_chunks = []
for line in output:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300, # 최대 청크 길이
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""], #  텍스트를 청크로 분할하는 데 사용되는 문자 목록
        chunk_overlap=200, # 인접한 청크 간에 중복되는 문자 수
    )
    chunks = text_splitter.split_text(line)
    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk, metadata={ "source": 'namu.pdf', "page": i}
        )
        doc_chunks.append(doc)

# 4. 인덱스 파이프라인 생성

In [ ]:
# HuggingFaceEmbeddings 클래스를 사용하여 임베딩 모델 로드
embed_model = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-nli"  # 한국어 문장 임베딩 모델
)

In [ ]:
# Chroma 데이터베이스 디렉토리 및 컬렉션 이름 설정
persist_directory = "./chromadb"
collection_name = "rscode_documents"

# 기존의 Chroma 인덱스를 로드하거나 없으면 새로 생성
try:
    index = Chroma(
        embedding_function=embed_model,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    # 컬렉션이 존재하면 문서 추가
    index.add_documents(doc_chunks)
except Exception as e:
    print(f"Error loading collection: {e}")
    # 컬렉션이 없으면 새로 생성하여 문서 추가
    index = Chroma.from_documents(
        documents=doc_chunks,
        embedding=embed_model,
        persist_directory=persist_directory,
        collection_name=collection_name
    )

# 변경된 내용을 디스크에 저장
index.persist()

# 검색기를 생성하여 유사도 검색 수행
retriever = index.as_retriever(search_kwargs={"k": 1})  # 검색 결과 상위 1개 반환

# 5. llm rag 파이프라인 생성

In [ ]:
modelName = model_id
# 모델 설정을 로드하여 GenerationConfig 객체 생성
gen_cfg = GenerationConfig.from_pretrained(modelName)
# 생성 설정 세부 조정
gen_cfg.max_new_tokens = 1024  # 생성할 최대 토큰 수
gen_cfg.temperature = 0.0000001  # 텍스트 생성의 무작위성을 낮추는 온도 설정
gen_cfg.return_full_text = True  # 생성된 텍스트 전체 반환
gen_cfg.do_sample = True  # 샘플링을 통해 텍스트 생성
gen_cfg.repetition_penalty = 1.11  # 반복 페널티 설정

# HuggingFace의 파이프라인을 사용하여 텍스트 생성 파이프라인 생성
# model.cuda()
pipe = pipeline(
    task="text-generation",  # 파이프라인 작업 유형 설정
    model=model,  # 사전 훈련된 언어 모델
    tokenizer=tokenizer,  # 모델에 맞는 토크나이저
    generation_config=gen_cfg,  # 생성 설정 적용
    # device=0  # 사용할 디바이스 설정 (기본 GPU를 사용할 경우 0으로 설정)
)

# LangChain의 HuggingFacePipeline을 사용하여 LLM 객체 생성
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# LLAMA 2 LLM에 권장되는 프롬프트 스타일을 사용합니다.
prompt_template = """
[INST] <>
You are a wise assistant. Answer questions accurately in Korean
<>
{context}

Question: {question} [/INST]
"""

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# RetrievalQA 체인을 생성합니다.
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,  # 사용할 LLM을 지정합니다.
    chain_type="stuff",  # 체인의 타입을 지정합니다.
    retriever=retriever,  # 문서를 검색할 때 사용할 retriever를 지정합니다.
    chain_type_kwargs={"prompt": prompt},  # 체인 타입에 대한 추가 인자를 지정합니다.
)

In [ ]:
# 질의 내용 설정
#query = "방탄소년단(BTS)의 콘셉트는?"
query = "폴리텍대학 광주캠퍼스 AI 융합과의 입학 조건은 무엇인가요?"
# 질의에 대한 답변을 생성합니다.
result = Chain_pdf.invoke(query)
result